In [9]:
import torch
import numpy as np

Check if torch package is installed

In [7]:
x = torch.rand(5, 3)

In [8]:
x

tensor([[0.2805, 0.0697, 0.3695],
        [0.3639, 0.2688, 0.1976],
        [0.7734, 0.3567, 0.5619],
        [0.3607, 0.6809, 0.9288],
        [0.8898, 0.9437, 0.8421]])

## Basics

### 1. Tensors

#### Initializing a tensor

In [10]:
x = [[1, 2], [3, 4]]

In [14]:
xt = torch.tensor(x)
print(type(xt))
xt

<class 'torch.Tensor'>


tensor([[1, 2],
        [3, 4]])

In [15]:
xnp = np.array(x)
xt = torch.from_numpy(xnp)
print(type(xt))
xt

<class 'torch.Tensor'>


tensor([[1, 2],
        [3, 4]])

In [17]:
x_ones = torch.ones_like(xt)
x_ones

tensor([[1, 1],
        [1, 1]])

In [20]:
x_rand = torch.rand_like(xt, dtype = torch.float)
x_rand

tensor([[0.7117, 0.4178],
        [0.9631, 0.3451]])

In [23]:
rand_t = torch.rand(5, 3)
ones_t = torch.ones(4, 5)
zeros_t = torch.zeros(1, 3)